<a href="https://colab.research.google.com/github/cedamusk/AI-N-ML/blob/main/SVM_for_Climate_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install earthengine-api geemap scikit-learn

In [15]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-climatechangeforedu')

In [16]:
import geemap
import sklearn

In [17]:
dataset = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').filter(
    ee.Filter.date('2018-05-01', '2018-05-03')
)
precipitation = dataset.select('precipitation')
precipitation_vis = {
    'min': 1,
    'max': 17,
    'palette': ['001137', '0aab1e', 'e7eb05', 'ff4a2d', 'e90000'],
}

m = geemap.Map()
m.set_center(17.93, 7.71, 2)
m.add_layer(precipitation, precipitation_vis, 'Precipitation')
m



Map(center=[7.71, 17.93], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [18]:
# Load the dataset
dataset = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR').first()

# Define visualization parameters
visualization = {
    'bands': ['temperature_2m'],
    'min': 250,
    'max': 320,
    'palette': [
        '000080', '0000d9', '4000ff', '8000ff', '0080ff', '00ffff',
        '00ff80', '80ff00', 'daff00', 'ffff00', 'fff500', 'ffda00',
        'ffb000', 'ffa400', 'ff4f00', 'ff2500', 'ff0a00', 'ff00ff',
    ]
}

# Create a map centered at the desired location
Map = geemap.Map(center=[45, 70], zoom=3)

# Add the layer
Map.addLayer(dataset, visualization, 'Air temperature (K) at 2m height')

# Display the map
Map


Map(center=[45, 70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

In [19]:
dataset=ee.ImageCollection('NOAA/CFSV2/FOR6H')\
          .filterDate('2018-03-01', '2018-03-14')

temperature_above_ground=dataset.select('Temperature_height_above_ground')

vis_params={
    'min':220.0,
    'max': 310.0,
    'paletter':['blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

Map=geemap.Map(center=[26.4, -88.6], zoom=1)

Map.addLayer(temperature_above_ground.mean(), vis_params, 'Temperature Above Ground')
Map

Map(center=[26.4, -88.6], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [20]:
# Load the dataset
dataset = ee.ImageCollection('RUB/RUBCLIM/LCZ/global_lcz_map/latest').mosaic()

# Define visualization parameters
visualization = {
    'bands': ['LCZ_Filter'],
    'min': 1,
    'max': 17,
    'palette': [
        '8c0000','d10000','ff0000','bf4d00','ff6600',
        'ff9955','faee05','bcbcbc','ffccaa','555555',
        '006a00','00aa00','648525','b9db79','000000',
        'fbf7ae','6a6aff'
    ]
}

# Create a map centered at the specified location
Map = geemap.Map(center=[51.44, 7.26], zoom=6)

# Add the dataset to the map
Map.addLayer(dataset, visualization, 'LCZ_Filter')

# Display the map
Map


Map(center=[51.44, 7.26], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [21]:
dataset = ee.Image('COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019').select(
    'discrete_classification'
)

m = geemap.Map()
m.set_center(-88.6, 26.4, 1)
m.add_layer(dataset, {}, 'Land Cover')
m

Map(center=[26.4, -88.6], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [22]:
dataset=ee.ImageCollection('MODIS/061/MCD18C2').filter(ee.Filter.date('2001-01-01', '2001-02-01'))

gmt_1200_par=dataset.select('GMT_1200_PAR')

import folium
from folium import plugins

map=folium.Map(location=[46.529, 6.746], zoom_start=2)

vis_params={
    'min': -236,
    'max':316,
    'palette':['0f17ff', 'b11406', 'f1ff23']
}

image=gmt_1200_par.mosaic()

map_id=image.getMapId(vis_params)
title_url=map_id['tile_fetcher'].url_format

folium.TileLayer(
    tiles=title_url,
    attr='Google Earth Engine',
    name='Total PAR at GMT 12:00',
    overlay=True,
    control=True
).add_to(map)

map

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

In [25]:
def initialize_earth_engine():
  try:
    ee.Initialize(project='ee-climatechangeforedu')
  except Exception as e:
    ee.Authenticate()
    ee.Initialize(project='ee-climatechangeforedu')

In [26]:
def fetch_climate_data():
  datasets={
      'precipitation':ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')
          .filter(ee.Filter.date('2018-01-01', '2018-12-31')),
      'temperature':ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR'),
      'land_cover':ee.ImageCollection('COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019')
  }
  return datasets

In [27]:
def get_african_climate_zones():
  return{
      'Sahara Desert': ee.Geometry.Rectangle(-20, 15, 40, 35),
      'Sahel':ee.Geometry.Rectangle(-20, 10, 40, 15),
      'Tropical Rainforest':ee.Geometry.Rectangle(10, -10, 40, 10),
      'Mediterranean': e.Geometry.Rectangle(-10, 30, 40, 37),
      'Subtropical':ee.Geometry.Rectangle(10, -35, 50, -10)
  }

In [ ]:
def extract_features(datasets, region_of_interest):
  precipitation=datasets["precipitation"].sum().clip(region_of_interest)
  precipitation_stats=precipitation.reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=region_of_interest,
      scale=5000
  )

  temperature=datasets['temperature'].select('temperature_2m')
  temp_mean=temperature.mean().clip(region_of_interest)
  temp_std=temperature.reduce(ee.Reducer.standardDeviation()).clip(region_of_interest)